# Visualization & Analysis of Chemical Data
This week we will look at a few ways to visualize and analyze chemical data. We already learned some ways to analyze chemical data in weeks 5 and 6 when we got familiar with RDKit. Visualizing through smart ways of plotting data is also a very important part of data analysis, which allows you to get a feeling of your data and to identify patterns.

Frst, let's get some housekeeping out of the way. We will use the same data as in weeks 5 and 6 from ChEMBL and filter it right away by excluding data with missing SMILES and USAN Definitions.

In [ ]:
# some imports
from rdkit import Chem
from rdkit.Chem import PandasTools
import pandas as pd
from pathlib import Path
import os

In [ ]:
# load data
current_file = Path(os.path.abspath(''))
csv_file = current_file.parent / "week_05" / "chembl_drugs.csv"
df = pd.read_csv(csv_file, sep= ";")
df.head()

If you get the following error in the next cell:
```
AttributeError: module 'pandas.io.formats.format' has no attribute 'get_adjustment'
```
Install pandas version 2.1 with the following command:
```
!pip install pandas==2.1
```

In [ ]:
# remove the rows with NaN values for SMILES and USAN Definition
df = df.dropna(subset=["Smiles", "USAN Definition"])
# add molecule objects to the df from SMILES
PandasTools.AddMoleculeColumnToFrame(df, smilesCol="Smiles", molCol="mol")

In [ ]:
df['USAN Definition'].unique()

For the purpose of this exercise we will filter out some specific drugs based on their USAN definition. We picked these four mode of actions: *antivirals*, *analgesics*, *beta-blockers* and *anti-inflammatory* drugs. We will keep any data point that has these words in their description and then make four new super classes out of them stored in `application`. You are free to choose different classes and test the influence on the results below. Depending on the homogeneity of the groups the clustering will give very different results.

In [ ]:
drug_applications = ['antivirals', 'analgesics', 'beta-blockers','anti-inflammatory']
# df matches one of the applications (not contains but exact match)
df = df[df['USAN Definition'].str.contains('|'.join(drug_applications), case=False, na=False)]
# add a column with the application combining all antivirals etc. into one class each
df['application'] = df['USAN Definition'].apply(lambda x: [app for app in drug_applications if app in x.lower()][0])

## Principal Component Analysis (PCA)
Principal Component Analysis (PCA) is a technique used to emphasize variation and bring out strong patterns in a dataset. It's often used to make data easy to explore and visualize. It is also used for dimensionality reduction, which is useful when you have a lot of features describing your data. PCA is a linear transformation method that finds the directions (principal components) that maximize the variance in the data. These directions are orthogonal to each other and form a new coordinate system in which the data can be represented. The first principal component is the direction in which the data varies the most, the second principal component is the direction in which the data varies the second most, and so on. We will plot the first two principal components of the data and thus reduce the dimnesionality of the data (going from n to 2 dimensions). For more details have a look at [this blog post](https://towardsdatascience.com/principal-component-analysis-pca-explained-visually-with-zero-math-1cbf392b9e7d).

We will perform PCA on Morgan fingerprints as features and make use of an the package `molplotly`, which allows plotting data interactively!

In [ ]:
!pip install molplotly
!pip install dash==2.10
!pip install scikit-learn

In [ ]:
# more imports
import numpy as np
from rdkit.Chem import AllChem, DataStructs
from sklearn.decomposition import PCA

### Carry out PCA algorithm

In [ ]:
def smi_to_fp(smi):
    '''
    Function to convert SMILES to Morgan fingerprint
    '''
    fp = # TODO convert SMILES to Morgan fingerprint (radius 2, 2048 bits)
    arr = np.zeros((0,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr

In [ ]:
# convert SMILES to Morgan fingerprints
df['fp'] = # TODO apply the function to the SMILES column
fps = np.array(df['fp'].tolist())

Look up [here](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) how to use `sklearn`'s PCA.

In [ ]:
# instantiate PCA class with 2 (principal) components
pca = # TODO code here
# fit and transform our data (fingerprints)
components = pca.fit_transform(fps.reshape(-1, 2048))
# store components in dataframe
df['PCA-1'] = # TODO code here to get first components
df['PCA-2'] = # TODO code here to get second components

### Plot the first two principal components
We already now how to plot data in a straight-forward way using `matplotlib`. While, it is very flexible we have very complex data and would like to see the influence of the molecular structure on the results. Therefore, we will look into the package[ `molplotly`](https://github.com/wjm41/molplotly/tree/main), which allows interactive plotting including much more information.

In [ ]:
import matplotlib.pyplot as plt

# plot the PCA components in a scatter plot
fig, ax = plt.subplots()
scatter = # TODO code here to plot PCA-1 vs PCA-2
ax.set_xlabel('PCA-1');
ax.set_ylabel('PCA-2');

This is the most basic implementation of such a PCA plot. If you like you can try to populate with more data such as colouring by drug types. In the next step we will look at how this is done with [`molplotly`](https://github.com/wjm41/molplotly/tree/main).

In [ ]:
import plotly.express as px
import molplotly

# TODO: fill in the names of the columns that are needed for the plot (as strings)

# instantiate the plot as done with the base package plotly
fig_pca = px.scatter(df, # data to pull from
                     x=, # x-axis
                     y=, # y-axis
                     color=, # column to color by (drug action)
                     title='PCA of morgan fingerprints', # title of the plot
                     labels={'color': XXXX}, # label for the color legend (same as "color")
                     width=1200,
                     height=800)

# populate with molecule information
app_pca = molplotly.add_molecules(fig=fig_pca,  # figure to populate
                                  df=df,  # dataframe with molecule information
                                  smiles_col=,  # column with SMILES
                                  title_col='Name',  # column to take the name for the molecule when hovering over it
                                  caption_cols=['USAN Definition'], # columns to take the additional captions from
                                  color_col=, # column to color by
                                  show_coords=False)

# show the plot: don't use the same port several times as it will overwrite the previous one
app_pca.run_server(mode='inline', port=8100, height=850)

Now, this plot is already much more informative. We can see the different drug groups and when hovering over the plot we can see the molecule the data point represents including the full USAN Definition and the name of the drug. We can also zoom in and out again!

This plot showed colouring based on discrete groups. What happens when we want to plot a continuous variable? We will look into this in the next section by calculating the logP of all the molecules.

In [ ]:
from rdkit.Chem import Descriptors

df["LogP"] = # TODO code here to calculate LogP


In [ ]:
import plotly.express as px
import molplotly

# TODO fill in the missing column names and color by logP this time

fig_pca = px.scatter(df,
                     x=,
                     y=,
                     color=,
                     title='PCA of morgan fingerprints',
                     labels={'color': XXXX},
                     width=1200,
                     height=800)

app_pca = molplotly.add_molecules(fig=fig_pca,
                                  df=df,
                                  smiles_col=,
                                  title_col='Name',
                                  caption_cols=['application'],
                                  color_col=,
                                  show_coords=False)

app_pca.run_server(mode='inline', port=8200, height=850)

This looks interesting too! However, we can see that there are some outliers that make the visulaization not so informative (and a s a side note also are not clustered very well with their group). Let's exclude those points and look at the plot again.

In [ ]:
# filter out logP below -5 and above 5
df_fil = df[(df['LogP'] > -5) & (df['LogP'] < 5)]

In [ ]:
# TODO repeat the same thing as above but with the filtered dataframe
# TODO don't forget to change the port number

In this last plot the differences in logP can be more easily observed. We can see that there is some trend that the clustering coincides with the logP. This is a very interesting observation but much more analysis would have to be done to get to the depths of this. This is a very simple example of how to use PCA to visualize and analyze chemical data. There are many more ways to use PCA and many more ways to visualize the data. We will look at some more in the next section.

## Clustering
Clustering data is a way to group data points that are similar to each other. There are many different clustering algorithms, and we will look at the k-means algorithm and the Taylor-Butina algorithm. Clsutering is mostly used when we don't know the underlying data structure and need it for analysis or data processing. Another use case is evaluating how meaningful our chosen representation (in our case the Morgan fingerprint) in the given context (here drug actions) is.

### K-means clustering
First, we will look into clustering using the k-means algorithm, which is simple and widely used. It tries to find cluster centers that are representative of certain regions of the data. The algorithm alternates between assigning data points to the nearest cluster center (based on some distance function) and updating cluster centers to be the mean of the data points that are assigned to them. We will use the [`KMeans`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) class from the `sklearn` package to perform k-means clustering.

In [ ]:
# even more imports
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
# define number of clusters based on number of drug actions
num_clusters = # TODO What is the number of broad drug actions

# TDOD instantiate KMeans object with num_clusters, a random state of 42 and n_init='auto'
km = # TODO code here

# fit data
fp_km = df['fp'].tolist()
# TODO code here to fit the data (fp_km)

# predict cluster number on our dat
clusters = # TODO code here

In [ ]:
# store in dataframe
df['cluster'] = clusters
# convert to strings (so that it recognizes this as discrete classes)
df['cluster'] = df['cluster'].astype(str)

In [ ]:
# plot the pca and color by cluster
fig_pca_clusters = # TODO code here to plot PCA-1 vs PCA-2 and color by cluster

app_pca_clusters = # TODO code here to add molecules to the plot

app_pca_clusters.run_server(mode='inline', port=8400, height=850)

Nice! If we compare to the previous plots, there certainly is some overlap with the true clusters. However, we also can see that it is far from perfect. This can have a multitude of reasons. Our drug classes contain heterogenous data, the Morgan fingerprints might not be the best representation of the data, or the clustering algorithm might not be the best choice. We will look into one of the classes next and see how diverse even one class is. Remember that we also comboned several USAN Definitions to simplify things. Thus, different molecule classes (defined by chemical motifs) were taken together.

#### Finding number of clusters
In the example above, we knew how many clusters we have in our data so we could help the algorithm by already supplying it with the right number. Now, we will look into how to find the right number of clusters in an unsupervised (without true labels) way and try to cluster the subgroup of all anti-inflammatory drugs. 

We will use the Silhouette score to find the optimal number of clusters. The Silhouette score is a measure of how similar an object is to its own cluster (cohesion) compared to other clusters (separation). The score ranges from -1 to 1, where a high value indicates that the object is well matched to its own cluster and poorly matched to neighboring clusters. We will calculate the Silhouette score for different numbers of clusters and choose the number of clusters that gives the highest score.

In [ ]:
# get all the anti-inflammatory drugs
antiinfl = df[df['application'] == 'anti-inflammatory'].copy()
antiinfl.head()

In [ ]:
# let's cluster the data with a random guess of numbers of clusters
num_clusters = 10

# †ODO code here to intantiate, fit and predict the clusters using KMeans as above

antiinfl['cluster'] = clusters
# convert to strings
antiinfl['cluster'] = antiinfl['cluster'].astype(str)

In [ ]:
# we need to repeat the PCA for the subselection of data
pca = PCA(n_components=2)
fsp_antinfl = np.array(antiinfl['fp'].tolist())
components = pca.fit_transform(fsp_antinfl.reshape(-1, 2048))
antiinfl['PCA-1'] = components[:, 0]
antiinfl['PCA-2'] = components[:, 1]

Show another interactive plot and color by newly determined cluster identity.

In [ ]:
fig_pca = # TODO code here to plot PCA-1 vs PCA-2 and color by cluster

app_pca = # TODO code here to add molecules to the plot

app_pca.run_server(mode='inline', port=8500, height=850)

Looks good! Let's have a more rational approach on determining the number of clusters. We will iterate over a range of 3 to 34 number of clusters and calculate the [`silhouette score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html). We will choose the number of clusters where the silhouette score is the highest. We can also plot the silhouette score over the cluster size.

In [ ]:
cluster_range = range(3,35)
score_list = []

for k in cluster_range:
    # instantiate KMeans class
    km = # TODO code here

    # fit and predict km based on the fingerprints
    cluster_labels = # TODO code here

    # calculate the silhouette score (between fP and cluster_labels)
    score = # TODO code here
    score_list.append([k,score])

In [ ]:
# plot silhouette score over number of clusters
score_df = pd.DataFrame(score_list,columns=['k','silhouette_score'])
fig, ax = plt.subplots()
ax.plot(score_df['k'],score_df['silhouette_score']);

In [ ]:
# let's determine the optimal cluster size programmatically
num_clusters_opt = score_df[score_df['silhouette_score'] == score_df['silhouette_score'].max()]['k'].values[0]
num_clusters_opt

We have the optimal cluster number determined to be 4. We can rerun the k-means algorithm with the optimal number of clusters.

In [ ]:
km_opt = KMeans(n_clusters=num_clusters_opt, random_state=42, n_init="auto")
clusters_opt = km_opt.fit_predict(antiinfl['fp'].tolist())
antiinfl['cluster_opt'] = clusters_opt
# convert to strings
antiinfl['cluster_opt'] = antiinfl['cluster_opt'].astype(str)

In [ ]:
fig_pca = # TODO code here to plot PCA-1 vs PCA-2 and color by "optimal" clusters

app_pca = # TODO code here to add molecules to the plot

app_pca.run_server(mode='inline', port=8600, height=850)

This looks pretty convincing. We can see that the clustering is much better than before. However, we can also see that there are still some data points that are still clustered in a weird way. This is a very common problem in clustering and can have many reasons but does not mean that anything went wrong with the clustering. As mentioned the before, the data representation has a huge influence but also the way that we visualize it is important. We will look into another clustering algorithm in the next section to see a last approach.

### Taylor-Butina clustering
Now. let's look at a different clustering algorithm, the Taylor-Butina clustering algorithm. This algorithm is based on the Tanimoto similarity between fingerprints so that all molecules in a cluster are more similar to the centroid than a pre-defined threshold but are potentially less similar to each other. This should result in small but homogenuous clusters. The steps to the algorithm are:
1. Calculate a fingerprint (e.g. Morgan fingerprint) for each molecule in the dataset.
2. Calculate the Tanimoto distance matrix
3. Cluster with the Taylor-Butina algorithm: 
    1. Centroids are initiated and sorted based on number of neighbors (Tanimoto similarity above (distance below) certain threshold).
    2. The centroid with the most neighbors gets all those assigned in its cluster.
    3. Move to the next biggest centroid group and assign.
    4. Repeat until no molecules with neighbors left (can have singletons - molecules without neighbors not assigned to a cluster)

With this algorithm one can get molecules in one cluster that are actually more similar to another cluster's centroid but were first assigned the biggest one. Thus, the choice of threshold is an important parameter to tune.

In [ ]:
from rdkit.ML.Cluster import Butina
from rdkit.Chem import rdMolDescriptors

In [ ]:
mols = antiinfl['mol'].tolist()

#### 1. Calculate fingerprints for all molecules in the dataset.
We have done this many times before. Let's to it again using a list comprehension in one line of code.

As a reminder - the structure of a list comprehension is:
```
[function(item) for item in iterable]
```


In [ ]:
fp_list =  #TODO code here (FP: radius 2 and 2048 bits)

In [ ]:
# check data type of the first element
assert type(fp_list[0]) == DataStructs.cDataStructs.ExplicitBitVect

#### 2. Calculate the Tanimoto distance matrix

We will use the [`DataStructs.BulkTanimotoSimilarity`](http://www.dalkescientific.com/writings/diary/archive/2020/10/02/using_rdkit_bulktanimotosimilarity.html) function from the `rdkit` package to calculate the Tanimoto similarity between all pairs of fingerprints. The Tanimoto similarity is a measure of how similar two sets are so we need to convert to distances by subtracting from 1 after.

This function calculates the Tanimoto similarity between all pairs of fingerprints in the dataset. The output is a list of lists where the element at position (i, j) is the Tanimoto similarity between the fingerprints of molecule i and molecule j. Try to get the Tanimoto distance between the first molecule to all the others (including intself).

In [ ]:
sims = # TODO code here
sims

We can see that the first element, as expected, gives similarity of one since it is the same molecule. The other values are between 0 and 1. The closer to 1 the more similar the molecules are. The function returns a list. We will build this list so that is a triangular distance matrix in the form of a list (so all rows are not the same length and concatenated). Write a loop to calculate the Tanimoto distance matrix where at every step you calculate the similarities between the current element and all *previous* elements.

In [ ]:
dists = []
nfps = len(fp_list)
for i in range(1,nfps):
    sims = # TODO code here: similarity between i-th molecule and all previous ones
    # convert similarity to dinstance (1-x)
    dists.extend([1-x for x in sims])
len(dists)

#### 3. Apply the Taylor-Butina algorithm

Check out the function docs [here](https://www.rdkit.org/docs/source/rdkit.ML.Cluster.Butina.html). The arguments that need to be passed are the following:
- `distances`: The distance matrix calculated in the previous step.
- `n`: The number of molecules in the dataset.
- `cutoff`: The Tanimoto similarity threshold to assign molecules to a cluster.
- `isDistData`: A boolean indicating whether the input is a distance matrix or similarity matrix.

In [ ]:
mol_clusters = Butina.ClusterData(
    data = # TODO code here
    nPts = # TODO code here
    distThresh = # TODO code here (pick a reasonable Tanimoto cutoff for clustering)
    isDistData = # TODO True or False?
    )
len(mol_clusters), type(mol_clusters)

Great! Now we have all the necessary parts to combine it all in one function that takes in as arguments the list of molecules and a given threshold and returns the list of cluster IDs (corresponding to the molecule list) and a the returned tuple of cluster indices sorted by cluster size.

In [ ]:
def butina_cluster(mol_list,cutoff=0.35):
    '''
    Carry out Tayloer-Butina algorithm.

    :param mol_list: list of RDKit molecules
    :param cutoff: Tanimoto cutoff value for clustering

    :return: list of cluster IDs for each molecule, dictionary with cluster ID -> [molecule indices]
    '''
    # 1. get fingerprint
    fp_list = # TODO code here to get fingerprints (radius 2, 2048 bits) from molecules in a list
    dists = []
    nfps = len(fp_list)

    # 2. get tanimoto distance matrix
    for i in range(1,nfps):
        sims = # TODO get BulkTanimotoSimilarity of the i-th fingerprint with all previous ones
        # convert similarity to dinstance (1-x)
        dists.extend([1-x for x in sims])

    # 3. apply Taylor-Butina algorithm (the rest happens under the hood)
    mol_clusters = # TODO code here
    cluster_id_list = [0]*nfps
    for idx,cluster in enumerate(mol_clusters,1):
        for member in cluster:
            cluster_id_list[member] = idx

    # get tuple sorted: cluster_id -> [molecule indices]
    mol_clusters = sorted(mol_clusters, key=len, reverse=True)

    return cluster_id_list, mol_clusters

In [ ]:
antiinfl['butina_cluster'], cluster_dict = # TODO apply the function

In [ ]:
# Plot the size of the clusters vs. their index
fig, ax = plt.subplots(figsize=(15, 4))
ax.set_xlabel("Cluster index")
ax.set_ylabel("Number of molecules")
ax.bar(range(1, len(cluster_dict) + 1), [len(c) for c in cluster_dict], lw=5);

We already addressed that the threshold determines how the clusters are built and is thus a parameter worth exploring. Before, we chose a random threshold (default value of 0.35) and can see that this clustering does not give good cluster sizes. In the following cell we will iterate over a range of thresholds and plot the cluster sizes sorted by size. Based on that we can select our optimal threshold. We want a smooth distribution while at the same time have as little singletons (clusters with one data point) as possible.

In [ ]:
for cutoff in np.arange(0.0, 1.0, 0.1):
    _, cluster_dict = butina_cluster(mols, cutoff=cutoff)
    fig, ax = plt.subplots(figsize=(15, 4))
    ax.set_title(f"Threshold: {cutoff:3.1f}")
    ax.set_xlabel("Cluster index")
    ax.set_ylabel("Number of molecules")
    ax.bar(range(1, len(cluster_dict) + 1), [len(c) for c in cluster_dict], lw=5)

Let's choose the best threshold based on the criteria we defined above. 

In [ ]:
mols = antiinfl['mol'].tolist()
best_cutoff = # TODO code here
antiinfl['butina_cluster'], cluster_dict = butina_cluster(mols, cutoff=best_cutoff)

Let's plot another interactive map as done before.

In [ ]:
fig_pca = px.scatter(antiinfl,
                     x="PCA-1",
                     y="PCA-2",
                     color='butina_cluster',
                     title='PCA of morgan fingerprints',
                     labels={'color': 'butina_cluster'},
                     width=1200,
                     height=800)

app_pca = molplotly.add_molecules(fig=fig_pca,
                                  df=antiinfl,
                                  smiles_col='Smiles',
                                  title_col='Name',
                                  caption_cols=['USAN Definition', 'cluster_opt', 'butina_cluster'],
                                  color_col='butina_cluster',
                                  show_coords=False)

app_pca.run_server(mode='inline', port=8700, height=850)


Because we have a lot of small clusters, the plot above is difficult to analyze. Let's visualize some molecules and see if the clustering makes sense to us from a chemistry perspective.

In [ ]:
from rdkit.Chem import Draw
print("Ten molecules from largest cluster:")
# Draw molecules
Draw.MolsToGridImage(
    [mols[i] for i in cluster_dict[0][:10]],
    molsPerRow=5,
)

In [ ]:
print("Ten molecules from second largest cluster:")
# Draw molecules
Draw.MolsToGridImage(
    [mols[i] for i in cluster_dict[1][:10]],
    molsPerRow=5,
)

In [ ]:
print("Ten molecules from third largest cluster:")
# Draw molecules
Draw.MolsToGridImage(
    [mols[i] for i in cluster_dict[2][:10]],
    molsPerRow=5,
)

I would argue that the three visualized clusters certainly show similarity between the molecules. Well done! This analysis is only one aspect and depending on the objective can be handled differently. But clustering and visualizing the data will help with getting a sense of what you are working with and if your data processing for a project can makes sense.